In [19]:
import cv2
import mtcnn.mtcnn
import numpy as np
from PIL import Image
import mysql.connector
from pathlib import Path
import matplotlib.pyplot as plt 
from keras.models import load_model


In [ ]:
#create database connection using myconnector
#connect database created in creat_db
face_db=mysql.connector.connect(
        host="localhost",
        user="root",
        password="Aimfame1@",
        database="face_db")

In [3]:
#define mtcnn detector
detector=mtcnn.MTCNN()
#load pre-trained keras model
model=load_model("facenet_keras.h5")

Instructions for updating:
Colocations handled automatically by placer.


/home/noor/.local/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [7]:
def preprocess(face_img):
    #preprocess input image
    
    #model expects input image of size 160*160
    #convert numpy array to image type
    face_img=Image.fromarray(face_img)
    #resize image
    face_img=face_img.resize((160, 160))
    #revert to numpy array
    face_img=np.asarray(face_img)
    print(face_img.shape)
    #standardize the pixels of sample
    mean,std=face_img.mean(),face_img.std()
    print(mean,std)
    face_img=(face_img-mean)/std
    #add fourth dimension to numpy array(number of images)
    sample=np.expand_dims(face_img,axis=0)
    print(sample.shape)
    return sample

In [5]:
path="/home/noor/Desktop/angelina_brad.jpeg"
img=plt.imread(path)
results = detector.detect_faces(img)

In [18]:
x1,y1,width,height=results[0]["box"]
#calculate (x2,y2) coordinate
x2,y2=x1+width,y1+height
cropped_img=img[y1:y2,x1:x2]
print(cropped_img.shape)

(98, 62, 3)


In [8]:
sample=preprocess(cropped_img)

(160, 160, 3)
151.45860677083334 54.120497600806594
(1, 160, 160, 3)


In [9]:
pred=model.predict(sample)
emb_vec=pred[0]        

In [ ]:
#capture video from camera
cap=cv2.VideoCapture(0)
while (cap.isOpened()):
    #capture frame by frame
    red,frame=cap.read()
    #detect the face using detector
    face=detector.detect_faces(frame)
    if len(face)==1 :
        x1,y1,width,height=face[0]["box"]
        x1,y1=abs(x1),abs(y1)
        #calculate (x2,y2) coordinate
        x2,y2=x1+width,y1+height
        face_img=image[y1:y2,x1:x2]
        sample=preprocess(face_img)
        #get embedding vector
        pred=model.predict(sample)
        emb_vec=pred[0]
        

In [21]:
#create cursor object
mycursor=face_db.cursor()

In [ ]:
#insert row into the table
query="INSERT INTO Registered_users (UniqueID,Embedding_Vector) VALUES (%s,%s)"
mycursor.execute(query,(unique_id,emb_vec))
#commit changes
face_db.commit()